In [4]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import hail as hl
hl.init()

FatalError: Hail has already been initialized, restart session or stop Hail to change configuration.

In [6]:
pwd

'/hail/large_vcf_analysis'

In [41]:
mt = hl.read_matrix_table('/hail/large_vcf_filtering_annotation/step6_output.mt')

In [42]:
mt.count()

(10742986, 302)

In [43]:
mt = mt.filter_cols(mt.s == 'WGS_81')

In [44]:
mt.count()

(10742986, 1)

In [45]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [46]:
mt = mt.drop('phenotypes', 'is_gnomad', 'gwas_p')

In [57]:
mt = mt.annotate_rows(info = hl.struct())

In [61]:
mt = mt.annotate_rows(info = mt.info.annotate(gnomad_v3_nfe = mt.gnomad_v3.v3_nfe.AF, 
                                              mutation_type = mt.gnomad_v3.most_severe_consequence, 
                                              gene = mt.gnomad_v3.transcript_consequences.gene_symbol,
                                              was_split = mt.was_split,
                                              cadd = mt.cadd))

In [62]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'AC': array<int32>
    'AF': array<float64>
    'AN': int32
    'a_index': int32
    'was_split': bool
    'cov_v3': struct {
        over_1: float32
    }
    'gnomad_v3': struct {
        v3_nfe: struct {
            AC: int32, 
            AF: float64, 
            AN: int32, 
            homozygote_count: int32
        }, 
        intergenic_consequences: array<struct {
            allele_num: int32, 
            consequence_terms: array<str>, 
            impact: str, 
            minimised: int32, 
            variant_allele: str
        }>, 
        most_severe_consequence: str, 
        motif_feature_consequences: array<struct {
            allele_num: int32, 
            consequence_terms: array<str>, 
            high_in

In [63]:
hl.export_vcf(mt, '/hail/data/example_vcfs/annotated.vcf.gz') 

2020-02-28 16:21:10 Hail: WARN: export_vcf: ignored the following fields:
    'AC' (row)
    'AF' (row)
    'AN' (row)
    'a_index' (row)
    'was_split' (row)
    'cov_v3' (row)
    'gnomad_v3' (row)
    'cadd' (row)
    'nearest_genes_20kb' (row)
2020-02-28 16:22:16 Hail: INFO: merging 603 files totalling 58.9M...
2020-02-28 16:22:17 Hail: INFO: while writing:
    /hail/data/example_vcfs/annotated.vcf.gz
  merge time: 199.246ms


In [66]:
mt = mt.drop('info')

In [67]:
hl.export_vcf(mt, '/hail/data/example_vcfs/naked.vcf.gz')

2020-02-28 16:23:24 Hail: WARN: export_vcf: ignored the following fields:
    'AC' (row)
    'AF' (row)
    'AN' (row)
    'a_index' (row)
    'was_split' (row)
    'cov_v3' (row)
    'gnomad_v3' (row)
    'cadd' (row)
    'nearest_genes_20kb' (row)
2020-02-28 16:23:24 Hail: WARN: export_vcf found no row field 'info'. Emitting no INFO fields.
2020-02-28 16:24:27 Hail: INFO: merging 603 files totalling 35.1M...
2020-02-28 16:24:27 Hail: INFO: while writing:
    /hail/data/example_vcfs/naked.vcf.gz
  merge time: 171.612ms
